In [111]:
# Initialisation
import os
import io
import requests
import json
import pandas as pd
import urllib.parse as url
import hvplot.pandas
import datetime as dt
import alpaca_trade_api as tradeapi
#import matplotlib.pyplot as plt
from dotenv import load_dotenv
from pathlib import Path

In [2]:
# Initialised
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension()

In [3]:
load_dotenv()
api_key = os.getenv("DOMAIN_API_KEY")
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca_api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [4]:
#%matplotlib inline

In [42]:
# REQUEST CALL DOMAIN.COM.AU: This api call will, given a string, finds the property address that closest matches the string
# and returns the address in full as a string, each address components in its own string, and a unique ID for the given address.
# We will return all this data in a df
def address_suggest(address):
    if not address:
        return
    address_url = url.quote(address)
    request_url = f"https://api.domain.com.au/v1/properties/_suggest?terms={address_url}&pageSize=1&api_key={api_key}"
    get_request = requests.get(request_url)

    # Data formatting
    data = get_request.json()
    # Extract data
    full_address = data[0]["address"]
    address_data = data[0]["addressComponents"]
    location_id = data[0]["id"]
    
    
    # Address_data is already a dict so we need to combine everything with update
    all_data = {
        "full_address": full_address,
        "id": location_id
    }
    all_data.update(address_data)
    # Put data into df
    # Df will have a single row. All data in their own columns with their own names
    address_df = pd.DataFrame([all_data])
    return address_df

In [7]:
# Address locators: this request will return a unique identifier for the following address levels:
# Address, street, suburb, postcode

def address_locators(address_df, level, building_type):
    if building_type == "unit":
        unit = address_df.iloc[0]["unitNumber"]
    else:
        unit = None
    street_num = address_df.iloc[0]["streetNumber"]
    street_name = address_df.iloc[0]["streetName"]
    street_type = address_df.iloc[0]["streetType"]
    suburb = address_df.iloc[0]["suburb"]
    post_code = address_df.iloc[0]["postCode"]
    state = address_df.iloc[0]["state"]
    
    # Check if building is actually a unit
    if building_type == "unit":
        request_url = f"https://api.domain.com.au/v1/addressLocators?searchLevel={level}&unitNumber={unit}&streetNumber={street_num}&streetName={street_name}&streetType={street_type}&suburb={suburb}&state={state}&postcode={post_code}&api_key={api_key}"

    else:
        request_url = f"https://api.domain.com.au/v1/addressLocators?searchLevel={level}&streetNumber={street_num}&streetName={street_name}&streetType={street_type}&suburb={suburb}&state={state}&postcode={post_code}&api_key={api_key}"


    get_request = requests.get(request_url)

    # Data formatting
    data = get_request.json()
    combined_id_data = {}
    for loc_id in data[0]["ids"]:
        combined_id_data.update({loc_id["level"]+" id": loc_id["id"]})
    
    id_df = pd.DataFrame([combined_id_data])
    
    return id_df

In [8]:
# Properties: Using a given property id provided by _suggest, this request gives an info dump on the characteristics of the property
# For now just returning all data since I do not know exactly what data is required
def properties(prop_id):
    request_url = f"https://api.domain.com.au/v1/properties/{prop_id}?api_key={api_key}"
    get_request = requests.get(request_url)
    data = get_request.json()
    return data

In [9]:
# Location profiles: this request gives an info dump of a given location based on the suburb id given. 
# ID must come from Address Locator request
def location_profiles(suburb_id):
    request_url = f"https://api.domain.com.au/v1/locations/profiles/{suburb_id}?api_key={api_key}"
    get_request = requests.get(request_url)
    data = get_request.json()
    return data

In [11]:
def suburb_price_chart(growth_df):
    return growth_df.hvplot.line(
        title = "Suburb's growth yearly",
        x = "Years",
        y = "Median Sold Price",
    ).opts(
        yformatter="%.0f"
    )

In [12]:
def suburb_sale_chart(growth_df):
    print(growth_df["Years"])
    return growth_df.hvplot.line(
        title = "Suburb's growth yearly",
        x = "Years",
        y = "Number of Sales",
    ).opts(
        yformatter="%.0f"
    )

In [141]:
# Top level of property section will do api calls on _suggest and address-locators, determine if property is a unit
# and will produce an info dump on the selected property
def property_top_level(address):
    if not address:
        return
    address_df = address_suggest(address)
    full_address = address_df.iloc[0]["full_address"]
    address_string = f"Showing data for the following address:  <br /> {full_address}"
    
    # Create a flag here on top level to indicate if property is a unit
    # unit number wont be a column in address_df if the property isn't a unit to begin with
    if address_df.iloc[0]["unitNumber"]:
        building_type = "unit"
    else:
        building_type = "house"
       
    # Get ids of address componenets
    id_df = address_locators(address_df, "address", building_type)
    
    # Get information on property. Need to know how many bedrooms property has
    property_data = properties(address_df.iloc[0]["id"])
    bedrooms = property_data["bedrooms"]
    info_string = f"This property is a {building_type}, with {bedrooms} bedrooms"
    
    # Get suburbs location profile
    location_data = location_profiles(id_df.iloc[0]["Suburb id"])
    categorised_location_data = location_data["data"]["propertyCategories"]
    
    # Single out the property type by building type and bedroom count
    for property_type in categorised_location_data:
        if property_type["bedrooms"] == bedrooms and property_type["propertyCategory"].lower() == building_type:
            specified_location_data = property_type
    
    # Code here uses specified location data for analysis
    growth_years = []
    growth_price = []
    growth_sales = []
    for entry in specified_location_data["salesGrowthList"]:
        growth_years.append(entry["year"])
        growth_price.append(entry["medianSoldPrice"])
        growth_sales.append(entry["numberSold"])
    
    growth_data = {"Years": growth_years,
                  "Median Sold Price": growth_price,
                  "Number of Sales": growth_sales}
    
    growth_df = pd.DataFrame(growth_data)
    #growth_df.set_index("Years", inplace=True)
    
    suburb_data_row = pn.Row(suburb_price_chart(growth_df), suburb_sale_chart(growth_df))
    
    test_column = pn.Column(info_string, suburb_data_row)
    
    
    return pn.Column(address_string, test_column, width = 1000)

In [169]:
def stock_top_level(stocks):
    if stocks is None:
        return
    stock_df = pd.read_csv(io.BytesIO(stocks), index_col="symbol", infer_datetime_format=True)
    stock_df["date"] = pd.to_datetime(stock_df["date"], infer_datetime_format=True)
    user_symbols = stock_df.index.tolist()
    start_date = stock_df["date"].tolist()
    start_date.sort()
    start_date = pd.Timestamp(start_date[0], tz="America/New_York").isoformat()
    end_date = (pd.Timestamp.today(tz="America/New_York")).isoformat()
    # Some notes about this api call: if the start and end date are outside of the 1000 day timeframe then it will return according to end_date
    # Dates must be in above format otherwise it wont work
    historical_data = alpaca_api.get_barset(
        user_symbols,
        "1D",
        start = start_date,
        end = end_date,
        limit = 1000
    ).df

    # We will now check if we have the purchase date in our dataset. If not, we will recursively pull data with the start date
    # we need but with an updated end date, and store any dataset we end up pulling
    extra_data_list = []
    top_date = pd.Timestamp(historical_data.index[0]).isoformat()
    while start_date != top_date:
        extra_data = alpaca_api.get_barset(
            user_symbols,
            "1D",
            start = start_date,
            end = top_date,
            limit = 1000
        ).df
        top_date = pd.Timestamp(extra_data.index[0]).isoformat()
        extra_data_list.append(extra_data)

    extra_data_list.append(historical_data)
    historical_df = pd.concat(extra_data_list, axis="rows", join="outer")
    historical_df.drop_duplicates(inplace=True)
        
    return pn.Pane(merged_df, width= 1000)

In [170]:
# Put everything together into this dashboard and serve it
title = "All In One Portfolio Summary and Analysis"

property_opening_message = """Please enter your full address  <br />
This program will find the address with the closest match to the address you provide it"""

property_opening_column = pn.Column(property_opening_message,
                                   interact(property_top_level, 
                                            address=widgets.TextInput(name="Your Address", placeholder="Please enter your address here")),
                                   width=600)

stock_opening_message = """Please upload a .csv file with your stock symbol, purchase date, and purchase amount  <br />
Please use the following column names in your csv: symbol, date, amount"""

stock_analysis_opening_column = pn.Column(stock_opening_message, 
                                          interact(stock_top_level, stocks=widgets.FileInput(accept=".csv")),
                                         width = 600)

tabs = pn.Tabs(
    ("Property Analysis", 
     property_opening_column),
    ("Stock Portfolio Analysis",
    stock_analysis_opening_column)
)   
dashboard = pn.Column(title, tabs)
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column(width=600)
            [0] Markdown(str)
            [1] Column
                [0] Column
                    [0] TextInput(name='Your Address', placeholder='Please enter y...)
                [1] Row
                    [0] Str(None, name='interactive02646')
        [1] Column(width=600)
            [0] Markdown(str)
            [1] Column
                [0] Column
                    [0] FileInput(accept='.csv')
                [1] Row
                    [0] Str(None, name='interactive02656')

In [16]:
property_top_level("20 winten drive glendenning")

0    2016
1    2017
2    2018
3    2019
4    2020
5    2021
Name: Years, dtype: int64


Column(width=1000)
    [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Curve)
            [1] HoloViews(Curve)

C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:16: DeprecatedWarning: get_barset is deprecated as of v1.0.0. Use get_bars instead
  app.launch_new_instance()
C:\Users\Kevin\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:29: DeprecatedWarning: get_barset is deprecated as of v1.0.0. Use get_bars instead


AAPL                                     \
                             open    high     low    close    volume   
time                                                                   
2022-02-11 00:00:00-05:00  172.33  173.08  168.04  168.440  75019260   
2022-02-14 00:00:00-05:00  167.37  169.58  166.56  169.130  64590666   
2022-02-15 00:00:00-05:00  170.99  172.95  170.25  172.800  51878759   
2022-02-16 00:00:00-05:00  171.82  173.34  170.05  172.550  50239943   
2022-02-17 00:00:00-05:00  171.15  171.72  170.74  171.305   5316171   

                               GOOG                                         
                               open       high       low    close   volume  
time                                                                        
2022-02-11 00:00:00-05:00  2771.890  2785.6650  2665.770  2683.20  1582463  
2022-02-14 00:00:00-05:00  2666.475  2723.3298  2666.040  2705.32  1041882  
2022-02-15 00:00:00-05:00  2747.830  2758.0000  2710.790  2728.48   979524  
2022-02-16 00:00:00-05:00  2727.970  2758.9200  2696.473  2749.76  1052603  
2022-02-17 00:00:00-05:00  2723.660  2736.7900  2721.000  2722.44    83349

2022-02-17 21:55:12.887708
2019-05-24 21:55:12.887708
2016-08-27 21:55:12.887708
2013-12-01 21:55:12.887708
2011-03-07 21:55:12.887708
